In [5]:
import os
os.system( '7z x train.csv.7z' )

0

In [6]:
#get strucutre of csv file by reading 1 row and deletting data from it
chunksize = 1
for chunk in pd.read_csv('train.csv', chunksize=chunksize):
    df= pd.DataFrame(chunk)
    break


In [9]:
df.dtypes

id               int64
date            object
store_nbr        int64
item_nbr         int64
unit_sales     float64
onpromotion    float64
dtype: object

In [10]:
# -*- coding: utf-8 -*-
import pandas as pd
from datetime import timedelta

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv('train.csv', usecols=[0,1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217) #Skip dates before 2016-08-01
                    )

In [11]:
import copy
train2=copy.deepcopy(train)

In [12]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales
0,86672216,2016-08-01,1,103520,3.0
1,86672217,2016-08-01,1,103665,2.0
2,86672218,2016-08-01,1,105574,7.0
3,86672219,2016-08-01,1,105575,13.0
4,86672220,2016-08-01,1,105577,2.0


In [13]:

from sklearn.model_selection import train_test_split

# train_trz, test_trz = train_test_split(tr_copy_z, test_size=0.2)

train_tr, test_tr = train_test_split(train, test_size=0.2)

In [48]:
test=test_tr.iloc[:,:].reset_index().iloc[:,1:5]

In [35]:
train = train_tr.iloc[:,1:].reset_index().iloc[:,1:]

In [51]:
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

In [52]:
#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
        ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
        ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

In [ ]:
ma_wk.head(5)

In [ ]:
ma_dw.head(5)

In [53]:
train.drop('dow',1,inplace=True)

# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()

In [ ]:
train2.head(5)

In [54]:
#del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date



In [ ]:
del u_dates, u_stores, u_items

In [55]:
#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais')

In [ ]:
ma_is.head(4)

In [ ]:
ma_is.head(4)

In [ ]:
train.head(5)
ma_is.head(4)

In [ ]:
train.head(5).median(axis=1)

In [56]:
#Going back these following days: addinga new column of moving average(Sales Mean)
#180,112,56,28,14,7,3

for i in [180,112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

In [ ]:
del tmp,tmpg,train

In [57]:
ma_is['mais']=ma_is.median(axis=1)

In [58]:
ma_is.reset_index(inplace=True)
ma_is.drop(list(ma_is.columns.values)[3:],1,inplace=True)

In [ ]:
ma_is.head()

In [75]:
train.head()

,date,store_nbr,item_nbr,unit_sales
0,2016-12-22,25,1909729,0.693147
1,2016-12-22,25,577743,0.000000
2,2016-12-22,25,819195,0.000000
3,2016-12-22,25,1345981,0.693147
4,2016-12-22,25,1146801,0.000000


In [ ]:
#test = pd.read_csv('test.csv', dtype=dtypes, parse_dates=['date'],skipinitialspace=True)

In [59]:
#Load test
#dtypes = {'id':'uint32', 'unit_sales':'float32'}
#test = pd.read_csv('test.csv', dtype=dtypes, parse_dates=['date'],skipinitialspace=True)
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

In [ ]:
del ma_is, ma_wk, ma_dw

In [63]:
test.head(4)

,id,date,store_nbr,item_nbr,dow,mais,mawk,madw,unit_sales
0,120597807,2017-06-30,34,838216,4,0.792140,1.287375,1.304755,1.231859
1,107034543,2017-02-22,3,2042210,2,1.122395,2.561990,3.181953,3.030934
2,103051814,2017-01-14,50,1060036,5,1.284510,1.671732,2.109395,4.057118
3,93283851,2016-10-08,37,165551,5,0.670922,1.337410,1.212678,0.837395


In [62]:
#Forecasting Test
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#50% more for promotion items
#test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

#test[['id','unit_sales']].to_csv('ma8dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')

In [76]:
train.head()

,date,store_nbr,item_nbr,unit_sales
0,2016-12-22,25,1909729,0.693147
1,2016-12-22,25,577743,0.000000
2,2016-12-22,25,819195,0.000000
3,2016-12-22,25,1345981,0.693147
4,2016-12-22,25,1146801,0.000000


In [82]:
train=train_tr

train['dow'] = train['date'].dt.dayofweek
train = pd.merge(train, ma_is, how='left', on=['item_nbr','store_nbr'])
train = pd.merge(train, ma_wk, how='left', on=['item_nbr','store_nbr'])
train = pd.merge(train, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])


#Forecasting Test
train['unit_sales'] = train.mais 
pos_idx = train['mawk'] > 0
train_pos = train.loc[pos_idx]
train.loc[pos_idx, 'unit_sales'] = train_pos['mais'] * train_pos['madw'] / train_pos['mawk']
train.loc[:, "unit_sales"].fillna(0, inplace=True)
train['unit_sales'] = train['unit_sales'].apply(pd.np.expm1) # restoring unit values 


In [67]:
#test- MAE
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_tr['unit_sales'], test['unit_sales']) 

5.398313

In [73]:
#test - RMSE
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(test_tr['unit_sales'], test['unit_sales']))

20.454134363224178

In [70]:
#test - R2
from sklearn.metrics import r2_score
r2_score(test_tr['unit_sales'], test['unit_sales'])

0.15841063827925095

In [74]:
#test - MAPE
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(test_tr['unit_sales'], test['unit_sales'])

59.125286340713501

In [83]:
train_tr['unit_sales'].shape

(31059859,)

In [84]:
train['unit_sales'].shape

(31059859,)

In [85]:
#train- MAE
from sklearn.metrics import mean_absolute_error
mean_absolute_error(train_tr['unit_sales'], train['unit_sales']) 

5.3726153

In [86]:
#train - RMSE
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(train_tr['unit_sales'], train['unit_sales']))

26.729664711438815

In [87]:
#train - R2
from sklearn.metrics import r2_score
r2_score(train_tr['unit_sales'], train['unit_sales'])

0.10501952478485799

In [88]:
#train - MAPE
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(train_tr['unit_sales'], train['unit_sales'])

58.603155612945557